In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import IFrame
InteractiveShell.ast_node_interactivity = "all"

# Existing company matching service

I needed to be able to evaluate against predictions made by the current company matching service that:

* Only made one match per record
* Only joined the most likely match for a record
* Had no bias to which table was being joined onto the other
* Reflected the "truest" belief of the service (using postcode)

What follows is an EDA done in SQL, which I'm pulling over with very few checks just so the code doesn't get lost.

I'm focusing on [Companies House company data](https://data.trade.gov.uk/datasets/a777d199-53a4-4d0a-bbbb-1559a86f8c4c#companies-house-company-data) and [UK exporters](https://data.trade.gov.uk/datasets/76fb2db3-ab32-4af8-ae87-d41d36b31265#uk-exporters).

`make dims` had been run to produce the dimension tables.

In [5]:
from cmf.data import utils as du
from dotenv import load_dotenv, find_dotenv
import os

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

True

## Dim table sizes

In [6]:
du.query(f"""
    select
        count(*)
    from
        {os.getenv("SCHEMA")}."hmrc_trade__exporters__dim";
""")

,count
0,254243


In [7]:
du.query(f"""
    select
        count(*)
    from
        "companieshouse"."companies";
""")

,count
0,5381225


## Lead Gen Experiments match method

### Left: companies house, right: exporters

In [ ]:
du.query_nonreturn(f"""
    drop table {os.getenv("SCHEMA")}.test_match_lr;
    create table {os.getenv("SCHEMA")}.test_match_lr as
        select distinct on (w_match.id, c_match.match_id)
            w_match.id as export_id,
            w_match.match_id as export_match_id,
            w_match.similarity as export_match_similarity,
            -- Ignore postcode, sum similarity
            (select sum(co::int) from unnest(regexp_split_to_array(left(w_match.similarity, 6), '')) as co) as match_sum_similarity,
            c_match.id as crn,
            c_match.match_id as company_match_id,
            c_match.similarity as company_match_similarity,
            -- Ignore postcode, sum similarity
            (select sum(co::int) from unnest(regexp_split_to_array(left(c_match.similarity, 6), '')) as co) as ch_sum_similarity
        from
            companieshouse.companies__match_ids w_match
        left join  
            hmrc.trade__exporters__match_ids c_match on
            w_match.match_id = c_match.match_id
        order by
            -- Order by similarity, take the top (see select statement)
            w_match.id, 
            c_match.match_id,
            (select sum(co::int) from unnest(regexp_split_to_array(left(c_match.similarity, 6), '')) as co) desc;
    """)

In [12]:
du.query(f"""
    select
        count(*)
    from
        {os.getenv("SCHEMA")}.test_match_lr;
""")

,count
0,5336353


In [13]:
du.query(f"""
    select
        count(*)
    from
        {os.getenv("SCHEMA")}.test_match_lr m
    inner join
        {os.getenv("SCHEMA")}."hmrc_trade__exporters__dim" d on
        d.id::text = m.crn;
""")

,count
0,69146


(of 254243)

In [14]:
du.query(f"""
    select
        count(*)
    from
        {os.getenv("SCHEMA")}.test_match_lr m
    inner join
        "companieshouse"."companies" d on
        d.id = m.export_id;
""")

,count
0,5157812


(of 5359637)

In [15]:
du.query(f"""
    select
        count(*)
    from
        {os.getenv("SCHEMA")}.test_match_lr lkp
    left join
        {os.getenv("SCHEMA")}."hmrc_trade__exporters__dim" l on
        l.id::text = lkp.crn
    left join
        "companieshouse"."companies" r on
        r.id = lkp.export_id
    where
        l.id is not null and r.id is not null
""")

,count
0,68500


### Left: exporters, right: companies house

In [ ]:
du.query_nonreturn(f"""
    drop table {os.getenv("SCHEMA")}.test_match_rl;
    create table {os.getenv("SCHEMA")}.test_match_rl as
        select distinct on (w_match.id, c_match.match_id)
            w_match.id as export_id,
            w_match.match_id as export_match_id,
            w_match.similarity as export_match_similarity,
            -- Ignore postcode, sum similarity
            (select sum(co::int) from unnest(regexp_split_to_array(left(w_match.similarity, 6), '')) as co) as match_sum_similarity,
            c_match.id as crn,
            c_match.match_id as company_match_id,
            c_match.similarity as company_match_similarity,
            -- Ignore postcode, sum similarity
            (select sum(co::int) from unnest(regexp_split_to_array(left(c_match.similarity, 6), '')) as co) as ch_sum_similarity
        from
            hmrc.trade__exporters__match_ids w_match
        left join  
            companieshouse.companies__match_ids c_match on
            w_match.match_id = c_match.match_id
        order by
            -- Order by similarity, take the top (see select statement)
            w_match.id, 
            c_match.match_id,
            (select sum(co::int) from unnest(regexp_split_to_array(left(c_match.similarity, 6), '')) as co) desc;
    """)

In [8]:
du.query(f"""
    select
        count(*)
    from
        {os.getenv("SCHEMA")}.test_match_rl;
""")

,count
0,3418561


In [9]:
du.query(f"""
    select
        count(*)
    from
        {os.getenv("SCHEMA")}.test_match_rl m
    inner join
        {os.getenv("SCHEMA")}."hmrc_trade__exporters__dim" d on
        d.id::text = m.export_id;
""")

,count
0,254243


(of 254243)

In [10]:
du.query(f"""
    select
        count(*)
    from
        {os.getenv("SCHEMA")}.test_match_rl m
    inner join
        "companieshouse"."companies" d on
        d.id::text = m.crn;
""")

,count
0,3273969


(of 5359637)

In [11]:
du.query(f"""
    select
        count(*)
    from
        {os.getenv("SCHEMA")}.test_match_rl lkp
    left join
        {os.getenv("SCHEMA")}."hmrc_trade__exporters__dim" l on
        l.id::text = lkp.export_id
    left join
        "companieshouse"."companies" r on
        r.id = lkp.crn
    where
        l.id is not null and r.id is not null
""")

,count
0,235820


## Why?

The algo gets to the top match on the right given what's on the left needs matching.

If all your exporters need matching, it can match them all -- even if some weren't its top pick.

If all your companies need matching, it can match them all -- even if some weren't its top pick.

Our method wants to _succeed_, not _evaluate_.

Recommend:

1. Write a new algorithm that isn't opinionated
2. OR choose the (flawed) one with CH on the left and leave this as something to iterate

## New method

* For two dim tables
* Connect company matching match tables, including things that weren't matched (full join)
* Connect in the two dim tables and show where we've successfully connected (because company matching matches FACT tables)
* Only one row is allowed per cluster. We prefer:
    * The highest score
    * Exists in one of the two dim tables
* If company matching scored a match not in our dim table highest, we drop it
    * It shouldn't do -- at worst it'll be tied for top match

In [ ]:
du.query_nonreturn(f"""
    drop table if exists {os.getenv("SCHEMA")}.ch_x_exp_eval;
    create table {os.getenv("SCHEMA")}.ch_x_exp_eval as (
        select distinct on (cluster)
            cluster,
            score,
            l_id,
            l_hit,
            r_id,
            r_hit
        from (
            select
                l_lkp.id as l_id,
                case 
                    when l.id is not null
                    then true
                    else false
                end as l_hit,
                r_lkp.id as r_id,
                case 
                    when r.id is not null
                    then true
                    else false
                end as r_hit,
                l_lkp.match_id as cluster,
                coalesce(
                    (
                        char_length(replace(l_lkp.similarity, '0', ''))
                        +
                        char_length(replace(r_lkp.similarity, '0', ''))
                    ),
                    0
                ) as score
            from
                "hmrc"."trade__exporters__match_ids" l_lkp
            full join
                companieshouse.companies__match_ids r_lkp on
                l_lkp.match_id = r_lkp.match_id
            left join
                _user_eaf4fd9a."hmrc_trade__exporters__dim" l on
                l.id::text = l_lkp.id
            left join
                "companieshouse"."companies" r on
                r.id = r_lkp.id	
        ) raw_matches
        order by
            cluster desc,
            score desc,
            l_hit desc,
            r_hit desc
    );
    """)

In [17]:
# all company matching entries from dim tables

du.query(f"""
    select
        count(*)
    from
        {os.getenv("SCHEMA")}.ch_x_exp_eval
    where
        l_hit = true or r_hit = true
""")

,count
0,188154


In [18]:
# all company matching entries from export dim table 

du.query(f"""
    select
        count(*)
    from
        {os.getenv("SCHEMA")}.ch_x_exp_eval
    where
        l_hit = true
""")

,count
0,188154


(of 254243)

In [19]:
# all company matching entries from company dim table

du.query(f"""
    select
        count(*)
    from
        {os.getenv("SCHEMA")}.ch_x_exp_eval
    where
        r_hit = true
""")

,count
0,175468


(of 5359637)

In [20]:
du.query(f"""
    select
        count(*)
    from
        {os.getenv("SCHEMA")}.ch_x_exp_eval
    where
        l_hit = true and r_hit = true
""")

,count
0,175468


## How do the approaches differ?

I set up three evaluation tables to compare and contrast.

* v1 is original method, CH on the left
* v2 is original method, exporters on the left
* v3 is the new method

In [ ]:
du.query_nonreturn(f"""
    drop table if exists {os.getenv("SCHEMA")}.match_test_v1;
    create table {os.getenv("SCHEMA")}.match_test_v1 as (
        select
            export_id as crn,
            crn as export_id,
            match_sum_similarity,
            ch_sum_similarity,
            lkp.export_match_id as cluster,
            r.company_name as ch_name,
            r.postcode as ch_pc,
            l.company_name as exp_name,
            l.postcode as exp_pc
        from
            {os.getenv("SCHEMA")}.test_match_lr lkp
        left join
            {os.getenv("SCHEMA")}."hmrc_trade__exporters__dim" l on
            l.id::text = lkp.crn
        left join
            "companieshouse"."companies" r on
            r.id = lkp.export_id
        where
            l.id is not null 
            and r.id is not null
    );
""")

In [ ]:
du.query_nonreturn(f"""
    drop table if exists {os.getenv("SCHEMA")}.match_test_v2;
    create table {os.getenv("SCHEMA")}.match_test_v2 as (
        select
            export_id,
            crn,
            match_sum_similarity,
            ch_sum_similarity,
            lkp.export_match_id as cluster,
            r.company_name as ch_name,
            r.postcode as ch_pc,
            l.company_name as exp_name,
            l.postcode as exp_pc
        from
            {os.getenv("SCHEMA")}.test_match_rl lkp
        left join
            {os.getenv("SCHEMA")}."hmrc_trade__exporters__dim" l on
            l.id::text = lkp.export_id
        left join
            "companieshouse"."companies" r on
            r.id = lkp.crn
        where
            l.id is not null 
            and r.id is not null
    );
""")

In [ ]:
du.query_nonreturn(f"""
    drop table if exists {os.getenv("SCHEMA")}.match_test_v3;
    create table {os.getenv("SCHEMA")}.match_test_v3 as (
        select
            lkp.*,
            r.company_name as ch_name,
            r.postcode as ch_pc,
            l.company_name as exp_name,
            l.postcode as exp_pc
        from
            {os.getenv("SCHEMA")}.ch_x_exp_eval lkp
        left join
            {os.getenv("SCHEMA")}."hmrc_trade__exporters__dim" l on
            l.id::text = lkp.l_id
        left join
            "companieshouse"."companies" r on
            r.id = lkp.r_id	
        where
            l_hit = true 
            and r_hit = true
    );
""")

### Agree

In [21]:
du.query(f"""
    select 
        count(*)
    from 
        {os.getenv("SCHEMA")}.match_test_v1 v1
    inner join
        {os.getenv("SCHEMA")}.match_test_v2 v2 on
        v1.export_id = v2.export_id
        and v1.crn = v2.crn
    inner join
        {os.getenv("SCHEMA")}.match_test_v3 v3 on
        v1.export_id = v3.l_id
        and v1.crn = v3.r_id;
""")

,count
0,64075


### Who does it better? v3 vs v1

In [22]:
# disagree: in v3, not v1

du.query(f"""
    select
        count(*)
        --*
    from
        {os.getenv("SCHEMA")}.match_test_v3 v3
    where not exists (
        select
            export_id,
            crn
        from
            {os.getenv("SCHEMA")}.match_test_v1 v1
        where
            v1.crn = v3.r_id
            and v1.export_id = v3.l_id
    );
""")

,count
0,111393


Does this accurately represent the belief of the matching service?

* e"2256473" to ch"03042765" via 3369780 -- appropriate. Two postcode-only exp matches, either as likely as the other. Including PC is the diff
* e"2407592" to ch"11911888" via 1013097 -- appropriate. Two equal matches, either as likely as the other. Order probably the diff
* e"2645274" to ch"01660807" via 2344457 -- appropriate. Two postcode-only exp matches, either as likely as the other. Including PC is the diff

In [23]:
# disagree: in v1, not v3

du.query(f"""
    select
        count(*)
        --*
    from
        {os.getenv("SCHEMA")}.match_test_v1 v1
    where not exists (
        select
            l_id,
            r_id
        from

            {os.getenv("SCHEMA")}.match_test_v3 v3
        where
            v1.crn = v3.r_id
            and v1.export_id = v3.l_id
    )
""")

,count
0,4645


Does this accurately represent the belief of the matching service?

* e"2925935" to ch"03512796" via 8133 -- Two equal matches, either as likely as the other. Order probably the diff
* e"11630588" to ch"387111" via 3092652 -- Two postcode-only exp matches, either as likely as the other. Including PC is the diff

What about CLUSTERS not being matched? That might prove a difference

In [24]:
du.query(f"""
    select
        count(*)
        --*
    from
        {os.getenv("SCHEMA")}.match_test_v1 v1
    where not exists (
        select
            v3.cluster
        from
            {os.getenv("SCHEMA")}.match_test_v3 v3
        where
            v1.cluster = v3.cluster
    );
""")

,count
0,322


322, all bad matches

1564656 doesn't exist in HMRC exporters match ids. How did this happen?

In [25]:
du.query(f"""
    select
        count(*)
        --*
    from
        {os.getenv("SCHEMA")}.match_test_v3 v3
    where not exists (
        select
            v1.cluster
        from
            {os.getenv("SCHEMA")}.match_test_v1 v1
        where
            v1.cluster = v3.cluster
    );
""")

,count
0,107070


100k rows, mostly looking solid

**🏆 V3 WINS**

### Who does it better? v3 vs v2

In [27]:
# disagree: in v3, not v2

du.query(f"""
    select
        count(*)
        --*
    from
        {os.getenv("SCHEMA")}.match_test_v3 v3
    where not exists (
        select
            export_id,
            crn
        from
            {os.getenv("SCHEMA")}.match_test_v2 v2
        where
            v2.crn = v3.r_id
            and v2.export_id = v3.l_id
    );
""")

,count
0,887


In [28]:
# disagree: in v2, not v3

du.query(f"""
    select
        count(*)
        --*
    from
        {os.getenv("SCHEMA")}.match_test_v2 v2
    where not exists (
        select
            l_id,
            r_id
        from
            {os.getenv("SCHEMA")}.match_test_v3 v3
        where
            v2.crn = v3.r_id
            and v2.export_id = v3.l_id
    );
""")

,count
0,61703


In [29]:
du.query(f"""
    select
        count(*)
        --*
    from
        {os.getenv("SCHEMA")}.match_test_v2 v2
    where not exists (
        select
            v3.cluster
        from

            {os.getenv("SCHEMA")}.match_test_v3 v3
        where
            v2.cluster = v3.cluster
    );
""")

,count
0,1422


1422, all bad matches

Cluster 2159702 doesn't exist in the exporters dataset. How has it been matched? Same for 100285

In [30]:
du.query(f"""
    select
        count(*)
        --*
    from
        {os.getenv("SCHEMA")}.match_test_v3 v3
    where not exists (
        select
            v2.cluster
        from
            {os.getenv("SCHEMA")}.match_test_v2 v2
        where
            v2.cluster = v3.cluster
    )
""")

,count
0,805


100k rows, good and bad matches.

* 18584 is wrong but uses postcode, and is a fair representation of the match system's belief
* 3140785 is right but uses postcode -- again, fair representation

**🏆 V3 WINS**